<article class="day-desc"><h2>--- Day 7: Laboratories ---</h2><p>You thank the cephalopods for the help and exit the trash compactor, finding yourself in the <a href="/2024/day/6">familiar</a> <a href="/2018/day/4">halls</a> of a North Pole research wing.</p>
<p>Based on the large sign that says "teleporter hub", they seem to be researching <em>teleportation</em>; you can't help but try it for yourself and step onto the large yellow teleporter pad.</p>
<p>Suddenly, you find yourself in an unfamiliar room! The room has no doors; the only way out is the teleporter. Unfortunately, the teleporter seems to be leaking <a href="https://en.wikipedia.org/wiki/Magic_smoke" target="_blank">magic smoke</a>.</p>
<p>Since this is a teleporter lab, there are lots of spare parts, manuals, and diagnostic equipment lying around. After connecting one of the diagnostic tools, it helpfully displays error code <code>0H-N0</code>, which apparently means that there's an issue with one of the <em>tachyon manifolds</em>.</p>
<p>You quickly locate a diagram of the tachyon manifold (your puzzle input). A tachyon beam enters the manifold at the location marked <code>S</code>; tachyon beams always move <em>downward</em>. Tachyon beams pass freely through empty space (<code>.</code>). However, if a tachyon beam encounters a splitter (<code>^</code>), the beam is stopped; instead, a new tachyon beam continues from the immediate left and from the immediate right of the splitter.</p>
<p>For example:</p>
<pre><code>.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
</code></pre>
<p>In this example, the incoming tachyon beam (<code>|</code>) extends downward from <code>S</code> until it reaches the first splitter:</p>
<pre><code>.......S.......
.......|.......
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
</code></pre>
<p>At that point, the original beam stops, and two new beams are emitted from the splitter:</p>
<pre><code>.......S.......
.......|.......
......|^|......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
</code></pre>
<p>Those beams continue downward until they reach more splitters:</p>
<pre><code>.......S.......
.......|.......
......|^|......
......|.|......
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
</code></pre>
<p>At this point, the two splitters create a total of only <em>three</em> tachyon beams, since they are both dumping tachyons into the same place between them:</p>
<pre><code>.......S.......
.......|.......
......|^|......
......|.|......
.....|^|^|.....
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
...............
</code></pre>
<p>This process continues until all of the tachyon beams reach a splitter or exit the manifold:</p>
<pre><code>.......S.......
.......|.......
......|^|......
......|.|......
.....|^|^|.....
.....|.|.|.....
....|^|^|^|....
....|.|.|.|....
...|^|^|||^|...
...|.|.|||.|...
..|^|^|||^|^|..
..|.|.|||.|.|..
.|^|||^||.||^|.
.|.|||.||.||.|.
|^|^|^|^|^|||^|
|.|.|.|.|.|||.|
</code></pre>
<p>To repair the teleporter, you first need to understand the beam-splitting properties of the tachyon manifold. In this example, a tachyon beam is split a total of <em><code>21</code></em> times.</p>
<p>Analyze your manifold diagram. <em>How many times will the beam be split?</em></p>
</article>

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from aocd import get_data

puzzle_input = get_data(day=7, year=2025)
puzzle_input[:20]

'....................'

In [3]:
sample_input = """.......S.......
...............
.......^.......
...............
......^.^......
...............
.....^.^.^.....
...............
....^.^...^....
...............
...^.^...^.^...
...............
..^...^.....^..
...............
.^.^.^.^.^...^.
..............."""
sample_input

'.......S.......\n...............\n.......^.......\n...............\n......^.^......\n...............\n.....^.^.^.....\n...............\n....^.^...^....\n...............\n...^.^...^.^...\n...............\n..^...^.....^..\n...............\n.^.^.^.^.^...^.\n...............'

In [4]:
from dataclasses import dataclass
from typing import Self


@dataclass(frozen=True, slots=True)
class TachyonManifoldDiagram:
    grid: tuple[str, ...]

    @classmethod
    def from_string(cls, input: str) -> Self:
        lines = tuple(input.splitlines())
        return cls(grid=lines)


sample_diagram = TachyonManifoldDiagram.from_string(sample_input)
sample_diagram

TachyonManifoldDiagram(grid=('.......S.......', '...............', '.......^.......', '...............', '......^.^......', '...............', '.....^.^.^.....', '...............', '....^.^...^....', '...............', '...^.^...^.^...', '...............', '..^...^.....^..', '...............', '.^.^.^.^.^...^.', '...............'))

In [5]:
def find_start(grid: tuple[str, ...]) -> int:
    for x, char in enumerate(grid[0]):
        if char == "S":
            return x
    raise ValueError("No start found")


find_start(sample_diagram.grid)

7

In [6]:
def get_splitters(row: str) -> set[int]:
    return {i for i, char in enumerate(row) if char == "^"}


get_splitters(sample_diagram.grid[-2])

{1, 3, 5, 7, 9, 13}

In [7]:
def step(beams: set[int], row: str) -> set[int]:
    splitters = get_splitters(row)
    unchanged_beams = beams - splitters
    active_splitters = beams & splitters
    split_beams = {i - 1 for i in active_splitters} | {i + 1 for i in active_splitters}
    return unchanged_beams | split_beams


step({6, 8}, sample_diagram.grid[4])

{5, 7, 9}

In [8]:
def count_splits_on_row(beams: set[int], row: str) -> int:
    splitters = get_splitters(row)
    return len(beams & splitters)


count_splits_on_row({6, 8}, sample_diagram.grid[4])

2

In [9]:
from itertools import accumulate


def count_splits(diagram: TachyonManifoldDiagram) -> int:
    start = find_start(diagram.grid)
    initial_beams = {start}
    rows = diagram.grid[1:]  # Exclude first row (contains S)
    beam_states = accumulate(
        rows,
        func=step,
        initial=initial_beams,
    )
    splits_per_row = (
        count_splits_on_row(beams, row) for beams, row in zip(beam_states, rows)
    )
    return sum(splits_per_row)


sample_part1 = count_splits(sample_diagram)
assert sample_part1 == 21
sample_part1

21

In [10]:
part1 = count_splits(TachyonManifoldDiagram.from_string(puzzle_input))
part1

1550

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>With your analysis of the manifold complete, you begin fixing the teleporter. However, as you open the side of the teleporter to replace the broken manifold, you are surprised to discover that it isn't a classical tachyon manifold - it's a <em><span title="Please disregard the wave interference patterns that would arise from the wave-particle duality of individual tachyon particles while repairing the manifold.">quantum</span> tachyon manifold</em>.</p>
<p>With a quantum tachyon manifold, only a <em>single tachyon particle</em> is sent through the manifold. A tachyon particle takes <em>both</em> the left and right path of each splitter encountered.</p>
<p>Since this is impossible, the manual recommends the many-worlds interpretation of quantum tachyon splitting: each time a particle reaches a splitter, it's actually <em>time itself</em> which splits. In one timeline, the particle went left, and in the other timeline, the particle went right.</p>
<p>To fix the manifold, what you really need to know is the <em>number of timelines</em> active after a single particle completes all of its possible journeys through the manifold.</p>
<p>In the above example, there are many timelines. For instance, there's the timeline where the particle always went left:</p>
<pre><code>.......S.......
.......|.......
......|^.......
......|........
.....|^.^......
.....|.........
....|^.^.^.....
....|..........
...|^.^...^....
...|...........
..|^.^...^.^...
..|............
.|^...^.....^..
.|.............
|^.^.^.^.^...^.
|..............
</code></pre>
<p>Or, there's the timeline where the particle alternated going left and right at each splitter:</p>
<pre><code>.......S.......
.......|.......
......|^.......
......|........
......^|^......
.......|.......
.....^|^.^.....
......|........
....^.^|..^....
.......|.......
...^.^.|.^.^...
.......|.......
..^...^|....^..
.......|.......
.^.^.^|^.^...^.
......|........
</code></pre>
<p>Or, there's the timeline where the particle ends up at the same point as the alternating timeline, but takes a totally different path to get there:</p>
<pre><code>.......S.......
.......|.......
......|^.......
......|........
.....|^.^......
.....|.........
....|^.^.^.....
....|..........
....^|^...^....
.....|.........
...^.^|..^.^...
......|........
..^..|^.....^..
.....|.........
.^.^.^|^.^...^.
......|........
</code></pre>
<p>In this example, in total, the particle ends up on <em><code>40</code></em> different timelines.</p>
<p>Apply the many-worlds interpretation of quantum tachyon splitting to your manifold diagram. <em>In total, how many different timelines would a single tachyon particle end up on?</em></p>
</article>

In [11]:
from collections import Counter


def passing_timelines(timelines: Counter[int], splitters: set[int]) -> Counter[int]:
    """Timelines that don't hit splitters — continue unchanged."""
    return Counter(
        {pos: count for pos, count in timelines.items() if pos not in splitters}
    )


passing_timelines(Counter({6: 2, 8: 3}), {8})

Counter({6: 2})

In [12]:
def splitting_timelines(timelines: Counter[int], splitters: set[int]) -> Counter[int]:
    """Timelines that hit splitters — need to be split left/right."""
    result: Counter[int] = Counter()
    for pos, count in timelines.items():
        if pos in splitters:
            result[pos - 1] += count
            result[pos + 1] += count
    return result


splitting_timelines(Counter({6: 2, 8: 3}), {8})

Counter({7: 3, 9: 3})

In [13]:
def step_quantum(timelines: Counter[int], row: str) -> Counter[int]:
    splitters = get_splitters(row)
    passing = passing_timelines(timelines, splitters)
    splitting = splitting_timelines(timelines, splitters)
    return passing + splitting


step_quantum(Counter({6: 2, 8: 3}), sample_diagram.grid[4])

Counter({7: 5, 9: 3, 5: 2})

In [14]:
def count_timelines(diagram: TachyonManifoldDiagram) -> int:
    start = find_start(diagram.grid)
    initial_timelines = Counter({start: 1})
    rows = diagram.grid[1:]  # Exclude first row (contains S)
    timeline_states = accumulate(
        rows,
        func=step_quantum,
        initial=initial_timelines,
    )
    final_timelines = list(timeline_states)[-1]
    return sum(final_timelines.values())


sample_part2 = count_timelines(sample_diagram)
assert sample_part2 == 40
sample_part2

40

In [15]:
part2 = count_timelines(TachyonManifoldDiagram.from_string(puzzle_input))
part2

9897897326778